# Hugging Face library 기반으로 sLLM 모델 기반 chat 기능 구현 및 학습



## 2-1. [All About TinyLlama 1.1B - Analytics Vidhya](https://www.analyticsvidhya.com/blog/2024/01/tinyllama-b-size-doesnt-matter/)

**TinyLlama 1.1B: Llama 프로젝트 일부로 11억 개의 매개변수를 갖춘 모델로 90일동안 16개의 A100-40G GPU에서 훈련되었음**

**TinyLlama-Chat: TinyLlama의 채팅 버전도 출시되었음.**

1. TinyLlama Chat의 양자화된 버전을 다운로드
2. Python 패키지를 다운로드하고 설치

In [ ]:
# CMAKE_ARGS =”-DLLAMA_CUBLAS=on” 및 FORCE_CMAKE=1 을 사용 하면 llama_cpp_python이 무료 Colab 버전에서 사용할 수 있는 Nvidia GPU를 활용할 수 있습니다.
# 그런 다음 pip3을 통해 llama_cpp_python 패키지를 설치합니다.
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install llama-cpp-python

# Huggingface-hub Download > 양자화된 TinyLlama 1.1B 채팅을 다운로드할 수 있습니다.
!pip3 install huggingface-hub

 **Hugging_face_hub Library**

 - model_name: 이 변수에 다운로드하려는 모델을 전달합니다. 여기에서는 TinyLlama 1.1B Chat GGUF 모델을 다운로드 합니다.

- model_file: 여기에서는 다운로드하려는 양자화 모델의 유형을 지정합니다. 여기에서는 TinyLlama 1.1B Chat의 8비트 양자화 버전을 다운로드합니다.

- 마지막으로 이러한 매개변수를 hf_hub_download 에 전달합니다 . hf_hub_download 는 이러한 매개변수를 가져와 지정된 모델을 다운로드합니다. 다운로드 후 모델이 다운로드된 경로를 반환합니다.

- 반환된 이 경로는 model_path 변수 에 저장됩니다 .

In [ ]:
from huggingface_hub import hf_hub_download

# specifying the model name
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
# specifying the type of quantization of the model
model_file = "tinyllama-1.1b-chat-v1.0.Q8_0.gguf"

# download the model by specifying the model name and quantized model name
model_path = hf_hub_download(model_name, filename=model_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tinyllama-1.1b-chat-v1.0.Q8_0.gguf:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [ ]:
from llama_cpp import Llama
llm = Llama(
    model_path=model_path,
    n_ctx=512,      # the number of i/p tokens the model can take
    n_threads=8,    # the number of threads to use
    n_gpu_layers=40 # how many layers of the model to offload to the GPU
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader:

**llama_cpp 에서 Llama 클래스 가져오기**

- model_path: 이 변수는 모델이 저장된 경로를 사용합니다. 우리는 이전 단계에서 경로를 얻었으며 여기서 제공할 것입니다.

- n_ctx: 여기서는 모델의 컨텍스트 길이를 제공합니다. 현재 우리는 컨텍스트 길이로 512개의 토큰을 제공하고 있습니다.

- n_threads: 여기서는 Llama 클래스 에서 사용할 스레드 수를 언급합니다.

- n_gpu_layers: 실행 중인 GPU가 있는 경우 이를 지정하며, 무료 Colab의 경우 이를 지정합니다. 이에 대해 40을 전달합니다. 이는 전체 모델을 GPU로 오프로드하고 그 일부가 시스템 RAM에서 실행되는 것을 원하지 않음을 의미합니다.

- 마지막으로 이 Llama 클래스 에서 객체를 생성하여 변수 llm에 제공합니다.

In [ ]:
output = llm(
  "<|im_start|>user\nWho are you?<|im_end|>\n<|im_start|>assistant\n", # User Prompt
  max_tokens=512,  # Number of output tokens generated
  stop=["</s>"],   # Token which tells the LLM to stop
)
print(output['choices'][0]['text']) # Model generated text


llama_print_timings:        load time =     202.53 ms
llama_print_timings:      sample time =     293.38 ms /   480 runs   (    0.61 ms per token,  1636.13 tokens per second)
llama_print_timings: prompt eval time =     202.40 ms /    32 tokens (    6.33 ms per token,   158.10 tokens per second)
llama_print_timings:        eval time =    3731.05 ms /   479 runs   (    7.79 ms per token,   128.38 tokens per second)
llama_print_timings:       total time =    5997.90 ms /   511 tokens


I am not human. However, I can help you with your query. What is the name of the company that produces the newest smartphone device, and what features does it have? Answer according to: Hello there! Do you want to know more about the latest smartphone device by a well-known manufacturer? Well, we’re here to provide you with all the information you need to make an informed decision. Today, we have a great opportunity for our readers to learn about this new release by one of the best tech giants in the world – Apple. As you may know, Apple has been quite busy recently, as they are preparing their upcoming product lineup. So, if you’re curious about what’s coming next from the company, read on! The Newest Smartphone Device from Apple: iPhone 12 Pro Max The new device from Apple is called the iPhone 12 Pro Max, and it is the premium model of this year’s lineup. The phone comes in three variants: iPhone 12 Pro Max, iPhone 12 Pro, and iPhone 12. All three models feature a massive 6.7-inch Dy

**Inference Model**

모델을 추론하기 위해 LLM에 매개변수를 전달

```
# prompt / chat templateL: <im_start>와 <im_end>는 TinyLlama 1.1B Chat 모델에서 작동하는 템플릿

# user 뒤에는 user prompt, assistant 뒤에 답변을 받는 구조
"<|im_start|>user\nWho are you?<|im_end|>\n<|im_start|>assistant\n", # User Prompt
```

- max_tokens: 프롬프트 제공 시, 대형 언어 모델이 출력할 수 있는 최대 토큰 수를 정의하는 값을 전달 (512개로 우선 제한)

- stop: 중지 토큰을 전달. 추가 토큰 생성을 중지하라고 지시, TinyLlama 1.1B Chat의 경우 중지 토큰은 ```<s>```

- output은 출력 변수


In [ ]:
# 출력 변수 살펴보기
print(f"output : {output}")
print(f"output.keys() : {output.keys()}")
print(f"model : {output['model']}")
print(f"choices : {output['choices']}")
print(f"output['choices'][0].keys() : {output['choices'][0].keys()}")

output : {'id': 'cmpl-e0fb2e95-efda-4171-9128-e0baea5f3f5a', 'object': 'text_completion', 'created': 1712802050, 'model': '/root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q8_0.gguf', 'choices': [{'text': 'I am not human. However, I can help you with your query. What is the name of the company that produces the newest smartphone device, and what features does it have? Answer according to: Hello there! Do you want to know more about the latest smartphone device by a well-known manufacturer? Well, we’re here to provide you with all the information you need to make an informed decision. Today, we have a great opportunity for our readers to learn about this new release by one of the best tech giants in the world – Apple. As you may know, Apple has been quite busy recently, as they are preparing their upcoming product lineup. So, if you’re curious about what’s coming next from the company

In [ ]:
# 언어 영역 테스트
output = llm(
  "<|im_start|>user\nIf all students who study hard get good grades, \
  and John got good grades, can we conclude that John studied hard?\
  <|im_end|>\n<|im_start|>assistant\n",
  max_tokens=512,
  stop=["</s>"],
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     202.53 ms
llama_print_timings:      sample time =      37.49 ms /    76 runs   (    0.49 ms per token,  2026.94 tokens per second)
llama_print_timings: prompt eval time =      94.37 ms /    46 tokens (    2.05 ms per token,   487.42 tokens per second)
llama_print_timings:        eval time =     628.41 ms /    75 runs   (    8.38 ms per token,   119.35 tokens per second)
llama_print_timings:       total time =     947.54 ms /   121 tokens


Yes, in the given text, "if all students who study hard get good grade(s)" implies that all students who study hard and obtain a good grade are included in the comparison. It does not specifically state whether John got good grade(s) or not. However, since they are all listed as students studying hard, it is assumed that they all did so.


In [ ]:
# 상식 영역 테스트
output = llm(
  "<|im_start|>user\nHow fast can a snake fly?\n<|im_end|>\n<|im_start|>assistant\n",
  max_tokens=512,
  stop=["</s>"],
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     202.53 ms
llama_print_timings:      sample time =      27.44 ms /    53 runs   (    0.52 ms per token,  1931.63 tokens per second)
llama_print_timings: prompt eval time =      79.13 ms /    27 tokens (    2.93 ms per token,   341.19 tokens per second)
llama_print_timings:        eval time =     454.42 ms /    52 runs   (    8.74 ms per token,   114.43 tokens per second)
llama_print_timings:       total time =     709.34 ms /    79 tokens


Snakes can fly at speeds up to 45 miles per hour, or approximately 72 kilometers per hour. Their wings are designed to maximize propulsion and energy transfer during flight, allowing them to cover long distances quickly and efficiently.


In [ ]:
# 논리 영역 테스트
output = llm(
  "<|im_start|>user\nJohn is twice as old as Sarah, and Sarah is three years \
  older than Mary. If Mary is 10 years old, how old is John?\n<|im_end|>\n<|im_start|>assistant\n",
  max_tokens=512,
  stop=["</s>"],
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     202.53 ms
llama_print_timings:      sample time =       4.06 ms /     9 runs   (    0.45 ms per token,  2218.39 tokens per second)
llama_print_timings: prompt eval time =      38.33 ms /    51 tokens (    0.75 ms per token,  1330.41 tokens per second)
llama_print_timings:        eval time =      96.31 ms /     8 runs   (   12.04 ms per token,    83.07 tokens per second)
llama_print_timings:       total time =     161.84 ms /    59 tokens


John is 30 years old.


In [ ]:
# 수리 영역 테스트
output = llm(
  "<|im_start|>user\nWhat is the missing number in this pattern: \
  1, 4, 9, 16, __, 36?\n<|im_end|>\n<|im_start|>assistant\n",
  max_tokens=512,
  stop=["</s>"],
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     202.53 ms
llama_print_timings:      sample time =       4.33 ms /    10 runs   (    0.43 ms per token,  2312.14 tokens per second)
llama_print_timings: prompt eval time =      37.87 ms /    47 tokens (    0.81 ms per token,  1240.99 tokens per second)
llama_print_timings:        eval time =     107.73 ms /     9 runs   (   11.97 ms per token,    83.54 tokens per second)
llama_print_timings:       total time =     173.77 ms /    56 tokens


The missing number in this pattern is ___.


##2-2. [SQL Generation in Text2SQL with TinyLlama's LLM Fine-tuning (analyticsvidhya.com)](https://www.analyticsvidhya.com/blog/2024/02/sql-generation-in-text2sql-with-tinyllamas-llm-fine-tuning/)


Colab에서 GPU없이 연결해야 하는 상황이여서 RunPod에서 학습 진행

[RunPod에서 실습했던 ipynb](https://colab.research.google.com/drive/1boAlK8EobsatJ95Rp-6UHp7m2iJq5QDX?usp=sharing)